In [ ]:
import xarray
import numpy as np
import sys

In [ ]:
import scipy
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
def subsample(data, sample_size):
    samples = list(zip(*[iter(data)]*sample_size))   # use 3 for triplets, etc.
    return map(lambda x:sum(x)/float(len(x)), samples)

In [ ]:
cyclone_ds = xarray.open_dataset("/g/data/x77/ob2720/partition/train/1999013S13164.nc", \
                                         engine='netcdf4', cache=True, chunks='auto', decode_cf=False)
cyclone_ds

In [ ]:
cyclone_ds = xarray.open_dataset("/g/data/x77/ob2720/partition/train/1999031N08092.nc", \
                                         engine='netcdf4', cache=True, chunks='auto', decode_cf=False)
cyclone_ds = cyclone_ds[dict(time=list(range(0, 8)),level=[1])][['v']]
cyclone_array = cyclone_ds.to_array().to_numpy()
cyclone_array = cyclone_array[:,:,:,40:120,40:120]
cyclone_array_new = cyclone_array[:,:,:,::4,::4].copy()
cyclone_array = cyclone_array.transpose((1,0,2,3,4))
cyclone_array_new = cyclone_array_new.transpose((1,0,2,3,4))
cyclone_array = cyclone_array.reshape((8,-1,80,80))
cyclone_array_new = cyclone_array_new.reshape((8,-1,20,20))
# print(cyclone_array[0])
print(cyclone_array_new[0])

In [ ]:
array = np.load("/g/data/x77/jm0124/np_cyclones_crop/4/train/1979066S15124-41.npy")
print(array[0])

In [ ]:
import seaborn
seaborn.heatmap(array[1,0,:,:]/1000)

In [ ]:
!ls /g/data/x77/jm0124/np_cyclones_crop/2/train

In [ ]:
print(array[::-1][0])
print(array[-1])

In [ ]:
from datasets import *

In [ ]:
!ls /g/data/x77/ob2720/partition/train/

## Training run with just eigenloss

In [1]:
import torch
import xarray
from datasets import *
from models import *
from dl_pipeline import *
import seaborn
import numpy as np
import matplotlib.pyplot as plt
import copy

In [2]:
model_dae = koopmanAE(32, steps=4, steps_back=4, alpha=8).to(0)

dataset, val_ds, test_ds = generate_example_dataset()
loader = torch.utils.data.DataLoader(dataset, batch_size=128, num_workers=8, pin_memory=True, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=128, num_workers=8, pin_memory=True, shuffle=True)

logging.info("Training DAE - only eigenloss")
model_dae, losses2, fwd_loss2, back_loss2, iden_loss2, cons_loss2 = train(model_dae, loader, len(dataset), 
                                                                          koopman=True, num_epochs=20, steps=4, eigen_penal=True,
                                                                          lamb=0.0, nu=0.0, eta=0.0, alpha=10, batch_size=128)

  8%|▊         | 100/1326.0234375 [00:25<06:36,  3.09it/s]

Minimum eigenvalue: 9.894254684448242
Loss: 199174.70403775302
Fwd loss: 198974.01960338245
Eigen loss: 200.68416152337585
[ 9.945325  +0.j         9.815344  +1.572093j   9.815344  -1.572093j
  9.25045   +3.5097108j  9.25045   -3.5097108j  8.729424  +4.7301354j
  8.729424  -4.7301354j  7.3391905 +6.7040367j  7.3391905 -6.7040367j
  5.4039884 +8.314118j   5.4039884 -8.314118j   4.701874  +8.74779j
  4.701874  -8.74779j    1.7904007 +9.783026j   1.7904007 -9.783026j
 -0.26314005+9.953278j  -0.26314005-9.953278j  -1.8264308 +9.773689j
 -1.8264308 -9.773689j  -3.317507  +9.360428j  -3.317507  -9.360428j
 -5.7262807 +8.124373j  -5.7262807 -8.124373j  -6.901529  +7.1153455j
 -6.901529  -7.1153455j -8.377587  +5.342816j  -8.377587  -5.342816j
 -9.273941  +3.5059586j -9.273941  -3.5059586j -9.927409  +0.5075786j
 -9.927409  -0.5075786j -9.933759  +0.j       ]


 15%|█▌        | 200/1326.0234375 [00:51<04:44,  3.96it/s]

Minimum eigenvalue: 9.832616806030273
Loss: 99741.11810026696
Fwd loss: 99542.1668680469
Eigen loss: 198.95110154271725
[ 9.896991  +0.j         9.771173  +1.5688425j  9.771173  -1.5688425j
  9.1945915 +3.482225j   9.1945915 -3.482225j   8.689117  +4.7076306j
  8.689117  -4.7076306j  7.3069453 +6.677584j   7.3069453 -6.677584j
  5.3792534 +8.275556j   5.3792534 -8.275556j   4.6717067 +8.697602j
  4.6717067 -8.697602j   1.785943  +9.7366295j  1.785943  -9.7366295j
 -0.25650978+9.908834j  -0.25650978-9.908834j  -1.8162987 +9.705771j
 -1.8162987 -9.705771j  -3.3041914 +9.317243j  -3.3041914 -9.317243j
 -5.6957817 +8.09383j   -5.6957817 -8.09383j   -6.8576727 +7.079128j
 -6.8576727 -7.079128j  -8.326261  +5.3168263j -8.326261  -5.3168263j
 -9.232416  +3.4852107j -9.232416  -3.4852107j -9.87582   +0.504132j
 -9.87582   -0.504132j  -9.886165  +0.j       ]


 23%|██▎       | 300/1326.0234375 [01:17<07:21,  2.33it/s]

...

 98%|█████████▊| 1301/1326.0234375 [05:33<00:09,  2.78it/s]

Minimum eigenvalue: 9.50451946258545
Loss: 15691.478319210672
Fwd loss: 15497.861503119466
Eigen loss: 193.61679657135127
[ 9.511403  +1.5197483j  9.511403  -1.5197483j  9.573063  +0.j
  8.895368  +3.3477116j  8.895368  -3.3477116j  8.455918  +4.5759473j
  8.455918  -4.5759473j  7.096046  +6.464215j   7.096046  -6.464215j
  5.2338676 +7.9664774j  5.2338676 -7.9664774j  4.564434  +8.465169j
  4.564434  -8.465169j   1.7523891 +9.43999j    1.7523891 -9.43999j
 -0.23493683+9.667204j  -0.23493683-9.667204j  -1.779789  +9.386561j
 -1.779789  -9.386561j  -3.1948967 +9.01066j   -3.1948967 -9.01066j
 -5.558243  +7.9392343j -5.558243  -7.9392343j -6.707609  +6.915497j
 -6.707609  -6.915497j  -8.094069  +5.1486163j -8.094069  -5.1486163j
 -8.972895  +3.383974j  -8.972895  -3.383974j  -9.614855  +0.4974873j
 -9.614855  -0.4974873j -9.641408  +0.j       ]


1327it [05:38,  3.92it/s]                                  


0. 120.15977810297137
Fwd loss: [118.6477460812604]
Back loss: [0.0]
Iden loss: [0.0]
Cons loss: [0.0]
Eigen loss: [1.5120318722974526]


PermissionError: [Errno 13] Permission denied: '/home/156/jm0124/kae-cyclones/saved_models/kae-model-continued-120.15977810297137.pt'

In [3]:
A = model_dae.dynamics.dynamics.weight.cpu().detach().numpy()
w, v = np.linalg.eig(A)
#w_abs = np.min(np.absolute(w)) 
np.min(np.absolute(w))

9.498313

In [1]:
import xarray
import numpy as np
import sys

In [2]:
import torch
import dataset_generation

In [18]:
dataset_generation.pendulum_to_ds(1,1)[0][0][0].size()

torch.Size([1, 64, 1])

In [58]:
import scipy
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import math

def simple_pendulum_deriv(x, t, m, g, l, F, c, omega): 
# The simple pendulum subject to zero damping and zero control input 
    nx = np.zeros(2)
    nx[0] = x[1]
    nx[1] = (1/m) * (F * math.sin(omega * t) - (m * g / l) * x[0] - c * nx[0])
    return nx

sol = odeint(simple_pendulum_deriv, y0=[0,0.5], t=np.linspace(0,15,200), args=(1,9.8,1,1,0.5,1))
y = sol[:,0]
y_dot = sol[:,1]

In [66]:
i = 0
sols = []
for start_pos in np.linspace(-math.pi,math.pi,10):
    for start_vel in np.linspace(-1,1, 10):
        sol = odeint(simple_pendulum_deriv, y0=[start_pos,start_vel], t=np.linspace(0,10,100), args=(1,9.8,1,1,0.5,1))
        sols.append(sol)